In [ ]:
import numpy as np
import requests
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
!pip install sodapy
from sodapy import Socrata
import pandas as pd

In [ ]:
def get_data(database_key, start_time, end_time,makor):
    # Unauthenticated client only works with public data sets. Note 'None'
    # in place of application token, and no username or password:
    makor = database_key

    # Example authenticated client (needed for non-public datasets):
    client = Socrata("data.cityofnewyork.us",
                    "u00Cxr2BfQIdhyzmtP61ghu8t",
                    username="bkatz7@mail.yu.edu",
                    password="Capstone2024")
    client.timeout = 60

    # First 2000 results, returned as JSON from API / converted to Python list of
    # dictionaries by sodapy.
    duration_query = "case(date_extract_hh(tpep_dropoff_datetime)>= date_extract_hh(tpep_pickup_datetime),(date_extract_hh (tpep_dropoff_datetime)*60 + date_extract_mm(tpep_dropoff_datetime) + date_extract_ss(tpep_dropoff_datetime)/60) - (date_extract_hh(tpep_pickup_datetime)*60 + date_extract_mm(tpep_pickup_datetime) + date_extract_ss(tpep_pickup_datetime)/60), date_extract_hh(tpep_dropoff_datetime)< date_extract_hh(tpep_pickup_datetime),(1440+ date_extract_hh(tpep_dropoff_datetime)*60+ date_extract_mm(tpep_dropoff_datetime) + date_extract_ss(tpep_dropoff_datetime)/60) - (date_extract_hh (tpep_pickup_datetime)*60+ date_extract_mm(tpep_pickup_datetime) + date_extract_ss(tpep_pickup_datetime)/60))"

    query_params = {
        "select": "pulocationid, AVG(total_amount), COUNT(*),"+
        " "
        f" AVG({duration_query}) AS avg_duration,"+
        " AVG(trip_distance),"+
        " FLOOR((date_extract_woy(tpep_pickup_datetime)*7*24*60+date_extract_dow(tpep_pickup_datetime)*24*60+date_extract_hh(tpep_pickup_datetime)*60+date_extract_mm(tpep_pickup_datetime))/30) AS interval, "+
        " ROUND(AVG(date_extract_woy(tpep_pickup_datetime)), 0) AS woy, "+
        " ROUND(AVG(date_extract_dow(tpep_pickup_datetime)), 0) AS dow, "+
        " ROUND(AVG(date_extract_y(tpep_pickup_datetime)), 0) as year, "+
        " ROUND(AVG(FLOOR((date_extract_hh(tpep_pickup_datetime)*60+date_extract_mm(tpep_pickup_datetime))/30)), 0) AS day_interval ",
        "where": f"tpep_pickup_datetime >= '{start_time}' AND tpep_pickup_datetime < '{end_time}' AND {duration_query}>1  AND {duration_query}<3*60 AND "+
        "total_amount>0 AND total_amount<300 AND "+
        "trip_distance>0.1 AND trip_distance<60 AND "+
        f" (total_amount/({duration_query})) <15 AND "+
        f" (total_amount/({duration_query})) > 0.20",

        "group": "interval, pulocationid",
        "order": "COUNT", #random
        "limit": 40000
    }
    #pulocationid,
    # Send the query and get the results
    results = client.get(makor, **query_params)
    # print(results)


    # Convert to pandas DataFrame
    results_df = pd.DataFrame.from_records(results)
    print(results_df)
    out = results_df
    """
    new_rows = []
    first_row = results_df.iloc[0]
    for zone in range(1, 266):
      for interval in range(0,48):
        if len(results_df[(results_df['pulocationid'] == str(zone)) & (results_df['day_interval'] == str(interval))]) > 0:
          new_row = {'pulocationid': zone, 'day_interval': interval, 'dow': first_row['dow'], 'year':first_row['year'], 'woy':first_row['woy'], 'COUNT':0, 'AVG_total_amount':0,'avg_duration':0,'AVG_trip_distance': 0, 'interval':17000  }
          new_rows.append(new_row)

        # create extension
    df_extended = pd.DataFrame(new_rows)
    # or columns=df.columns if identical columns

    # concatenate to original
    out = pd.concat([results_df, df_extended])
    """
    #print(out)
    return out


In [ ]:
# Build LSTM model
"""
model = Sequential([
    #, input_shape=(300, 7)
    LSTM(64, input_shape = (1, 7)),  # Assuming 7 features
    Dense(4)  # Output layer with 4 neurons for 4 features to predict
])
"""
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout



input_shape = (16,)
#drop out and batch normalization
# Create the DNN model
model = Sequential([
    Dense(64, activation='relu', input_shape=input_shape),


    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(4, activation = 'relu')  # Output layer with 1 neurons for predicting 1 continuous features
])
# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mean_squared_error'])

In [ ]:
from datetime import datetime

def get_old_date(year, week_of_year, day_of_week):
  # Convert to date
  #soda dow 0-6 sunday-saturday
  #iso calendar 1-7 monday-sunday
  if(day_of_week == 0):
    day_of_week = 7
  date_obj = datetime.fromisocalendar(year, week_of_year, day_of_week)

  # Convert date object to string
  date_string = date_obj.strftime('%Y-%m-%d')
  print("Date:", date_string)
  return date_obj


In [ ]:

db_keys = {2016: "uacg-pexx", 2017:"biws-g3hs", 2018:"t29m-gskq", 2019: "2upf-qytp", 2020:"kxp8-n2sj"}

In [ ]:
def preprocess_data(df):

  last_dow_value = int(df.loc[1, 'dow'])
  last_woy_value = int(df.loc[1, 'woy'])
  last_year_value = int(df.loc[1, 'year'])-1


  last_date_time = (get_old_date(last_year_value, last_woy_value, last_dow_value))
  start_date = last_date_time
  end_date = start_date + timedelta(days=1)
  start_time = start_date.strftime('%Y-%m-%dT%H:%M:%S')
  end_time = end_date.strftime('%Y-%m-%dT%H:%M:%S')
  last_year_data = get_data(db_keys[last_year_value], start_time, end_time, db_keys[last_year_value])
  result = pd.merge(df, last_year_data, on=['pulocationid', 'day_interval', 'dow', 'woy'],suffixes=('_current', '_last_year'))
  print("last year data")
  print(last_year_data)
  print("result")
  print(result)
  X = result[["pulocationid"]].astype(int)


  X = result[["dow"]].astype(int)
  all_dow = ["dow_" + str(i) for i in range(0,7)]
  X_dow_encoded = pd.DataFrame(0, index=X.index, columns=all_dow)
  # Update values based on the actual data in each batch
  for dow in all_dow:
      if int(dow[4:]) in X['dow'].unique():
          X_dow_encoded[dow] = (X['dow'] == int(dow[4:])).astype(int)

  X = result.drop("dow", axis= 1)
  X = pd.concat([result, X_dow_encoded], axis=1)






  y = X[["AVG_total_amount_current", "COUNT_current", "avg_duration_current", "AVG_trip_distance_current"]]
  X = X.drop(["AVG_total_amount_current", "COUNT_current", "avg_duration_current", "AVG_trip_distance_current", "interval_current", "interval_last_year", "year_last_year"], axis = 1)


  y = y.astype(float)
  print("X")
  print(X)
  print("y")
  print(y)
  # Y_array = np.concatenate((Y_array, y.values), axis=0)

  return X, y


In [ ]:
# 1. Save the model in Colab


# 2. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')



Mounted at /content/drive/


In [ ]:
# Pull the model from google drive. continue training from where we left off


# 2. Specify the path to the model file in your Google Drive
model_path = '/content/drive/My Drive/models/taxi_model_2018-04-06_Month.h5'  # Adjust the path accordingly

# 3. Copy the model file from Google Drive to Colab
!cp "$model_path" /content/

# 4. Load the model into your Python environment
from tensorflow.keras.models import load_model
loaded_model = load_model('/content/taxi_model_2018-04-06_Month.h5')

In [ ]:
from datetime import datetime, timedelta
import random

month_to_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

month_to_days_dict = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31,6: 30, 7: 31, 8: 31,9: 30,10: 31,11: 30,12: 31}
months = list(range(1, 13))
months = [2, 11, 10, 4, 3, 9, 1, 5, 7, 8, 6, 12]
#random.shuffle(months)
months = [10, 4, 3, 9, 1, 5, 7, 8, 6, 12]


mekoros = ["biws-g3hs","t29m-gskq","2upf-qytp"]
mekoros = ["t29m-gskq","2upf-qytp"]

data_2020= "kxp8-n2sj"

#get a random month
for month in months:
  #train the model on each month from each year before moving to next month
  print("Month:", month)
  #get a list of how many days are in this given month and then shuffle them
  days = list(range(1,month_to_days_dict[month]))
  random.shuffle(days)
  print("monthly fit")
  Monthly_X_tensor = tf.zeros(shape=(0, 16))
  Monthly_Y_tensor = tf.zeros(shape=(0, 4))
  for day in days:
      start_date = datetime(2018, month, day, 0, 0)  # Assuming March 2024
      end_date = start_date + timedelta(days=1)
      start_time = start_date.strftime('%Y-%m-%dT%H:%M:%S')
      end_time = end_date.strftime('%Y-%m-%dT%H:%M:%S')
      print("Start Date:", start_date)
      print("End Date:", end_date)

      df = get_data(db_keys[2018], start_time, end_time,db_keys[2018])
      X, Y = preprocess_data(df)
      # Convert X and Y to TensorFlow tensors
      X_tensor = tf.convert_to_tensor(X.values, dtype=tf.float32)
      Y_tensor = tf.convert_to_tensor(Y.values, dtype=tf.float32)

      Monthly_X_tensor = tf.concat([Monthly_X_tensor, X_tensor], axis=0)
      Monthly_Y_tensor = tf.concat([Monthly_Y_tensor, Y_tensor], axis=0)

      #model.fit(X_tensor, Y_tensor, epochs = 100)

  model.fit(Monthly_X_tensor, Monthly_Y_tensor, epochs = 100)
  model.save(f'/content/taxi_model_{str(start_date)[:10]}.h5')
  model_name = f'taxi_model_{str(start_date)[:10]}.h5'
  destination_folder = '/content/drive/My Drive/models/'
  !cp /content/{model_name} "$destination_folder"


# df
# #df[df["interval"]=='3255']
# df[df["interval"]=='3264']

Streaming output truncated to the last 5000 lines.
1      6         2018           10        19.8783333333333333               6   
2      6         2018           10         8.5500000000000000               2   
3      6         2018           14        35.1500000000000000               1   
4      6         2018           14        15.8700000000000000               1   
...   ..          ...          ...                        ...             ...   
4368   6         2018            5        14.3505806451612903             620   
4369   6         2018            1        15.3038563049853372             682   
4370   6         2018            2        14.8023617339312407             669   
4371   6         2018            4        14.7636350777934936             707   
4372   6         2018            3        14.1873333333333333             690   

       avg_duration_last_year AVG_trip_distance_last_year interval_last_year  \
0     26.39444444444444444445          9.3833333333333333 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Streaming output truncated to the last 5000 lines.
1      3         2018           12        25.8000000000000000               1   
2      3         2018           12        23.2166666666666667               3   
3      3         2018           14         9.3000000000000000               1   
4      3         2018           44         6.5500000000000000               2   
...   ..          ...          ...                        ...             ...   
4164   3         2018           43        13.2608500000000000             400   
4165   3         2018           46        15.4353185595567867             361   
4166   3         2018           36        13.3516309719934102             607   
4167   3         2018           40        13.7214868105515588             417   
4168   3         2018           41        14.9753378378378378             592   

       avg_duration_last_year AVG_trip_distance_last_year interval_last_year  \
0     27.28333333333333333334         10.7000000000000000 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Streaming output truncated to the last 5000 lines.
...   ..          ...          ...                        ...             ...   
4199   4         2018           40        15.0502782324058920             611   
4200   4         2018           39        15.0497520661157025             605   
4201   4         2018           43        13.4278019801980198             505   
4202   4         2018           40        14.3653992395437262             526   
4203   4         2018           39        14.2415243902439024             492   

       avg_duration_last_year AVG_trip_distance_last_year interval_last_year  \
0     17.48333333333333333333          6.9000000000000000               3252   
1     18.25000000000000000000         14.0300000000000000               3216   
2     15.45238095238095238095          6.1857142857142857               3216   
3      9.90000000000000000000          2.7000000000000000               3216   
4     23.48333333333333333334          4.7900000000000000     

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Streaming output truncated to the last 5000 lines.
0            6.3000000000000000               1   3.60000000000000000000   
1           34.3100000000000000               2  18.35833333333333333334   
2           10.3000000000000000               1  10.21666666666666666667   
3           18.4750000000000000               4  17.25833333333333333333   
4           13.0500000000000000               6  13.90555555555555555556   
...                         ...             ...                      ...   
4070        14.6641836734693878             294  14.70090702947845804989   
4071        11.5729610389610390             385  11.62341991341991341991   
4072        14.7051242236024845             322  12.95967908902691511387   
4073        15.9194117647058824             187  15.43734402852049910873   
4074        11.6119829424307036             469  12.09914712153518123667   

     AVG_trip_distance_last_year  dow_0  dow_1  dow_2  dow_3  dow_4  dow_5  \
0         1.14000000000000000000  

KeyboardInterrupt: 

In [ ]:
model.summary()
model = loaded_model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1088      
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 batch_normalization_1 (Bat  (None, 32)                128       
 chNormalization)                                                
                                                                 
 dropout_1 (Dropout)         (None, 32)                0

In [ ]:
start_date = datetime(2019, 3, 15, 0, 0)  # Assuming March 2024
end_date = start_date + timedelta(days=1)
start_time = start_date.strftime('%Y-%m-%dT%H:%M:%S')
end_time = end_date.strftime('%Y-%m-%dT%H:%M:%S')
print("Start Date:", start_date)
print("End Date:", end_date)
# Calculate 15 minutes before and after the current interval
df = get_data(db_keys[2019], start_time, end_time, "")

Start Date: 2019-03-15 00:00:00
End Date: 2019-03-16 00:00:00
     pulocationid     AVG_total_amount COUNT             avg_duration  \
0              56  13.3000000000000000     1  14.46666666666666666666   
1              72  16.3000000000000000     1  19.58333333333333333334   
2              72  20.3000000000000000     1  27.33333333333333333333   
3              70  29.7600000000000000     1  29.73333333333333333333   
4              76  27.3100000000000000     1  31.71666666666666666666   
...           ...                  ...   ...                      ...   
5691          162  17.1529898989898990   495  13.70952861952861952862   
5692           79  16.2337349397590361   498  13.04424364123159303882   
5693          161  17.6153996101364522   513  13.72069525666016894087   
5694          142  14.8920388349514563   515  10.84000000000000000000   
5695           79  16.9094746716697936   533  13.93205128205128205128   

        AVG_trip_distance interval woy dow  year day_interval

In [ ]:
X_test, Y_test = preprocess_data(df)
X_tensor = tf.convert_to_tensor(X_test.values, dtype=tf.float32)
Y_tensor = tf.convert_to_tensor(Y_test.values, dtype=tf.float32)
# Evaluate the model on the test data
y_pred = model.predict(X_tensor)

print(y_pred)

# # The evaluation result typically contains metric
# print("Evaluation result:", evaluation_result)

Date: 2018-03-16
     pulocationid     AVG_total_amount COUNT             avg_duration  \
0             138  60.1500000000000000     2  27.55000000000000000000   
1             189  14.3000000000000000     2  19.98333333333333333334   
2             190  21.9600000000000000     2  23.43333333333333333333   
3             257  33.3000000000000000     2  33.36666666666666666667   
4             217  15.8000000000000000     2  22.20000000000000000000   
...           ...                  ...   ...                      ...   
5229          237  11.9445989304812834  1122  12.18089720736779560309   
5230          236  12.5716608391608392  1144  12.90783799533799533800   
5231          230  14.8987108013937282  1148  14.82848432055749128920   
5232           79  15.8712953367875648  1158  14.68054116292458261370   
5233          230  15.7053650254668930  1178  15.24518958687040181098   

        AVG_trip_distance interval woy dow  year day_interval  
0     16.6000000000000000     3943  11   5

In [ ]:
print(y_pred[2000:2020])
print(Y_tensor[2000:2020])

NameError: name 'y_pred' is not defined

In [ ]:
import numpy as np
import pandas as pd

# Generate some sample sales data
dates = pd.date_range(start='2022-01-01', end='2022-01-31')
sales = np.random.randint(100, 1000, size=len(dates))

# Create a DataFrame to hold the data
sales_data = pd.DataFrame({'Date': dates, 'Sales': sales})

# Define the number of previous days to consider for prediction
num_previous_days = 7  # You can adjust this based on your requirements

# Prepare the input features
X_train = []
for i in range(num_previous_days, len(sales_data)):
    X_train.append(sales_data['Sales'].values[i - num_previous_days:i])

# Convert to numpy array
X_train = np.array(X_train)

# Check the shape of the data
print("X_train shape:", X_train)


In [ ]:
from flask import Flask, request, jsonify
from datetime import timedelta
from datetime import datetime

from sodapy import Socrata
import pandas as pd

app = Flask(__name__)


from datetime import datetime
def break_up_date(date_obj):


  # Get week of the year
  week_of_year = date_obj.isocalendar()[1]

  # Get day of the week (Monday is 1 and Sunday is 7)
  day_of_week = date_obj.isoweekday()

  return week_of_year, day_of_week


def get_old_date(year, week_of_year, day_of_week):
  # Convert to date
  date_obj = datetime.fromisocalendar(year, week_of_year+1, day_of_week)

  # Convert date object to string
  date_string = date_obj.strftime('%Y-%m-%d')
  print("Date:", date_string)
  return date_obj

def get_data(start_time, end_time):
    # Unauthenticated client only works with public data sets. Note 'None'
    # in place of application token, and no username or password:


    # Example authenticated client (needed for non-public datasets):
    client = Socrata("data.cityofnewyork.us",
                    "u00Cxr2BfQIdhyzmtP61ghu8t",
                    username="bkatz7@mail.yu.edu",
                    password="Capstone2024")

    # First 2000 results, returned as JSON from API / converted to Python list of
    # dictionaries by sodapy.
    query_params = {
        "select": "pulocationid, AVG(total_amount), COUNT(*),  AVG( (date_extract_hh(tpep_dropoff_datetime)- date_extract_hh(tpep_pickup_datetime))*60 +(date_extract_mm(tpep_dropoff_datetime)-date_extract_mm(tpep_pickup_datetime))) AS avg_duration,"+
        " AVG(trip_distance)",
        "where": f"tpep_pickup_datetime >= '{start_time}' AND tpep_pickup_datetime < '{end_time}'",

        "group": "pulocationid",
        "order": "AVG_total_amount"
        #"limit": 40000
    }

    # Send the query and get the results
    results = client.get("biws-g3hs", **query_params)
    # print(results)


    # Convert to pandas DataFrame
    results_df = pd.DataFrame.from_records(results)

    return results_df
@app.route('/historical', methods=['GET'])
def historical():
    # Get the datetime parameter from the query string
    print(request.args)
    datetime_param = request.args.get('datetime')

    # Convert string to datetime object
    date_obj = datetime.fromisoformat(datetime_param)
    time_obj = date_obj.time()
    week, day = break_up_date(date_obj)
    old_date = get_old_date(2017, week, day)
    look_up_date = datetime.combine(old_date, time_obj)
    print(look_up_date)


    start_time = look_up_date - timedelta(minutes=15)
    end_time = look_up_date + timedelta(minutes=15)
    start_time = start_time.strftime('%Y-%m-%dT%H:%M:%S')
    end_time = end_time.strftime('%Y-%m-%dT%H:%M:%S')
    print(start_time)
    print(end_time)

    data = get_data(start_time, end_time)



    # Return the response as JSON
    data.set_index('pulocationid', inplace = True)
    print(data)
    return data.to_json(orient='index')

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
from datetime import datetime, timedelta

# Define the start and end date for the month
start_date = datetime(2017, 3, 1, 0, 0)  # Assuming March 2024
end_date = datetime(2017, 4, 1, 0, 0)    # Assuming April 2024

# Loop through every 15-minute interval of the month
current_date = start_date
full_day_X = []
full_day_y = []
X_array = np.empty((0, 272))
y_array = np.empty((0, 4))